# Programming Lab 1 

## Handout 6 

## Lara Schmalenstroer (s0laschm@uni-bonn.de)

### Ex.1 (10pts) Reading and writing sequences in FASTA format
#### (a) (1pt) Read a single sequence
Write a function single_fasta_sequence(filename) that reads a sequence given in FASTA format from a file containing a single sequence. The function should return a tuple containing the headerline as first element (without the starting > and without the end-of-line character) and the sequence as a string as second element (without any end-of-line  characters). Note, that a sequence can span multiple lines but you should return the sequence as a single string.

In [1]:
def single_fasta_sequence(filename):
    with open(filename,'r') as seq:
#         title=seq.read().split('\n')[0][1:]
        fasta=seq.read().split('\n')
    title=fasta[0][1:]
    fasta_seq=''.join(fasta[1:])
    return (title,fasta_seq)

In [2]:
hd,seq=single_fasta_sequence('ecoli-genome.fna')

In [3]:
print(hd)

gi|556503834|ref|NC_000913.3| Escherichia coli str. K-12 substr. MG1655, complete genome


In [4]:
print(len(seq))

4641652


#### (b) (2pts) Reading multiple sequences from a file
A FASTA file can contain more than one sequence. Write a function fasta_list(filename) that reads all sequences from a FASTA file and returns a list of tuples, each tuple containing the header as the first and the sequence as the second element. Note, that a function written this way would normally first read the complete data contained in the FASTA file and return all the data in a single datastructure, i.e., a list of tuples.

In [5]:
def fasta_list(filename):
    with open(filename,'r') as seqs:
        fasta=seqs.read().split('>')[1:]
    list_fasta=[]
    for i in range(len(fasta)):
        s=fasta[i].split('\n')
        title=s[0]
        fasta_seq=''.join(s[1:])
        list_fasta.append((title,fasta_seq))
    return list_fasta

In [6]:
fasta_list('ecoli-proteome.faa')

[('gi|16127995|ref|NP_414542.1| thr operon leader peptide [Escherichia coli str. K-12 substr. MG1655]',
  'MKRISTTITTTITITTGNGAG'),
 ('gi|16127996|ref|NP_414543.1| fused aspartokinase I and homoserine dehydrogenase I [Escherichia coli str. K-12 substr. MG1655]',
  'MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNHLVAMIEKTISGQDALPNISDAERIFAELLTGLAAAQPGFPLAQLKTFVDQEFAQIKHVLHGISLLGQCPDSINAALICRGEKMSIAIMAGVLEARGHNVTVIDPVEKLLAVGHYLESTVDIAESTRRIAASRIPADHMVLMAGFTAGNEKGELVVLGRNGSDYSAAVLAACLRADCCEIWTDVDGVYTCDPRQVPDARLLKSMSYQEAMELSYFGAKVLHPRTITPIAQFQIPCLIKNTGNPQAPGTLIGASRDEDELPVKGISNLNNMAMFSVSGPGMKGMVGMAARVFAAMSRARISVVLITQSSSEYSISFCVPQSDCVRAERAMQEEFYLELKEGLLEPLAVTERLAIISVVGDGMRTLRGISAKFFAALARANINIVAIAQGSSERSISVVVNNDDATTGVRVTHQMLFNTDQVIEVFVIGVGGVGGALLEQLKRQQSWLKNKHIDLRVCGVANSKALLTNVHGLNLENWQEELAQAKEPFNLGRLIRLVKEYHLLNPVIVDCTSSQAVADQYADFLREGFHVVTPNKKANTSSMDYYHQLRYAAEKSRRKFLYDTNVGAGLPVIENLQNLLNAGDELMKFSGILSGSLSYIFGKLDEGMSFSEATTLAREMGYTEPDPRDDLSGMDVARKLLILARETGRELELADIEIEPVLPAEFNAEGDVAAFMANLSQLDDLFAARVA

#### (c) (2pts) Using generators to read entries from FASTA files

In [7]:
def fasta_sequences(filename):
    with open(filename,'r') as file:
        sequence=''
        while True:
            line = file.readline()
            if sequence=='':
                sequence+=line
            elif sequence!='' and line[0]!='>':
                sequence+=line
            else:
                sequence=sequence.split('\n')
                header=sequence[0][1:]
                seq=''.join(sequence[1:])
                sequence=line
                yield(header,seq)
            if not line:
                break

In [8]:
test=fasta_sequences('ecoli-proteome.faa')

In [9]:
next(test)

('gi|16127995|ref|NP_414542.1| thr operon leader peptide [Escherichia coli str. K-12 substr. MG1655]',
 'MKRISTTITTTITITTGNGAG')

#### (d) (1pt) Testing 
Test the functions written in (b) and (c) on ecoli-proteome.faa. Read the Fasta file and print the header and length of the shortest, longest as well as the last aminoacid sequence of the file

In [10]:
#function from (b)

list_seq=fasta_list('ecoli-proteome.faa')
maximum=len(list_seq[0][1])
minimum=len(list_seq[0][1])
for header, seq in list_seq:
    if len(seq)>maximum:
        maximum=len(seq)
        header_max=header
    if len(seq)<minimum:
        minimum=len(seq)
        header_min=header
        
print('Longest sequence: {}, length: {}'.format(header_max,maximum))
print('-----------------------------------------------------------')
print('Shortest sequence: {}, length: {}'.format(header_min,minimum))
print('-----------------------------------------------------------')
print('Last sequence: {}, length: {}'.format(list_seq[-1][0],len(list_seq[-1][1])))

Longest sequence: gi|145698281|ref|NP_416485.4| putative adhesin [Escherichia coli str. K-12 substr. MG1655], length: 2358
-----------------------------------------------------------
Shortest sequence: gi|16129226|ref|NP_415781.1| trp operon leader peptide [Escherichia coli str. K-12 substr. MG1655], length: 14
-----------------------------------------------------------
Last sequence: gi|16132220|ref|NP_418820.1| predicted methyltransferase [Escherichia coli str. K-12 substr. MG1655], length: 228


#### (e) (2 pts) Writing FASTA sequences 

In [13]:
def write_fasta(file,header,sequence):
    n=70
    seq=[]
    i=0
    while i < len(sequence):
        if i+n < len(sequence):
            seq.append(sequence[i:i+n])
        else:
            seq.append(sequence[i:len(sequence)])
        i += n
    if header[0]!='>':
        header='>'+header
    file.write(header+'\n')
    for line in seq:
        file.write(line+'\n')
    return

#### (f) (2pts) Creating a module: Putting things together. 
Put the definitions of the previous functions in a single file named fastatools.py. If you would run this script by itself you would observe no effect because all it does is define the functions. However, these functions might be actually useful as part of another script. Or, if you run it from the notebook the functions can be used from within the notebook. You can use the import statement, which you already know from Python standard modules such as math.

#### Ex.2 (2pts) Complementary DNA
Write a script cdna.py that takes two command line parameters. The first is an input file containing a DNA sequence in FASTA format. Read the sequence and generate the cDNA (complementary DNA strand) and write it to the file given as second parameter. You might want to add "cDNA" to the header of the output file. Note: that nucleotide sequences are written in 5' to 3' direction. Your output should also give the sequence in the 5' to 3' direction! Note: You can test your program on the small sample file ecoli-genome-sample.fna before running it on the bi gecoli-genome.fna file.

In [15]:
%run cdna.py ecoli-genome-sample.fna output-test-ecoli.txt

In [16]:
%run cdna.py ecoli-genome.fna output-ecoli.txt

### Ex.3 (10pts) Finding genes in prokaryotes: Open Reading Frames

In [32]:
%run orffinder.py ecoli-genome-sample.fna out_test.txt

[('frame 3', 'ATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAA', ' | :30-98 ')]
[('frame 2', 'ATGGTTTTTTCAATCATCGCCACCAGGTGGTTGGTGATTTTGGCGGGGGCAGAGAGGACGGTGGCCACCTGCCCCTGCCTGGCATTGCTTTCCAGAATATCGGCAACACGCAGAAAACGTTCTGCATTTGCCACTGATGTACCGCCGAACTTCAACACTCGCATGGTTGTTACCTCGTTACCTTTGGTCGAAAAAAAAAGCCCGCACTGTCAGGTGCGGGCTTTTTTCTGTGTTTCCTGTACGCGTCAGCCCGCACCGTTACCTGTGGTAATGGTGATGGTGGTGGTAATGGTGGTGCTAATGCGTTTCATGGATGTTGTGTACTCTGTAATTTTTATCTGTCTGTGCGCTATGCCTATATTGGTTAAAGTATTTAGTGACCTAAGTCAATAA', ' | :c500-108 '), ('frame 3', 'ATGTACCGCCGAACTTCAACACTCGCATGGTTGTTACCTCGTTACCTTTGGTCGAAAAAAAAAGCCCGCACTGTCAGGTGCGGGCTTTTTTCTGTGTTTCCTGTACGCGTCAGCCCGCACCGTTACCTGTGGTAA', ' | :c364-230 '), ('frame 3', 'ATGTTGTGTACTCTGTAA', ' | :c187-170 ')]


In [40]:
%run orffinder.py ecoli-genome.fna out_test_full.txt

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Ex.4 (8pts) Assessing the quality of the ORF approach
#### (a) (2pts) Get gene sequence positions from headers

In [50]:
from fastatools import fasta_list
import re

def get_sequence_positions(fasta_file):
    list_seq=fasta_list(fasta_file)
    headers=[]
    positions={}
    for header,seq in list_seq:
        headers.append(header)
    for header in headers:
        x = re.search(r"[0-9]+[-][0-9]+", header)
        match=x.group(0)
        match=match.split('-')
        positions[int(match[0])]=int(match[1])
    return positions

In [51]:
get_sequence_positions('out_test.txt')

{30: 98, 500: 108, 364: 230, 187: 170}

In [52]:
get_sequence_positions('ecoli-orfs.ffn')

{30: 98,
 500: 108,
 187: 170,
 190: 255,
 364: 230,
 337: 2799,
 374: 487,
 512: 562,
 654: 517,
 734: 621,
 644: 703,
 813: 748,
 872: 813,
 887: 882,
 917: 1075,
 1154: 1131,
 1409: 1200,
 1235: 1273,
 1393: 1382,
 1625: 1470,
 1547: 1666,
 1821: 1648,
 1673: 1747,
 1878: 1828,
 1871: 1948,
 2106: 1945,
 1988: 1968,
 2027: 2224,
 2240: 2148,
 2288: 2259,
 2351: 2389,
 2477: 2608,
 2624: 2650,
 2654: 2791,
 2782: 2738,
 2781: 2840,
 2802: 2788,
 2801: 3733,
 2891: 2805,
 2853: 2873,
 2883: 2906,
 2930: 2907,
 3213: 2998,
 3217: 3086,
 3132: 3179,
 3432: 3217,
 3306: 3476,
 3418: 3371,
 3483: 3566,
 4162: 3512,
 3734: 5020,
 4119: 3811,
 3864: 3875,
 4049: 3918,
 3927: 4088,
 4161: 4187,
 4278: 4201,
 4250: 4218,
 4293: 4313,
 4380: 4300,
 4326: 4340,
 4716: 4763,
 4867: 4760,
 4842: 5003,
 5014: 4961,
 5031: 5014,
 5059: 5142,
 5088: 5237,
 5129: 5137,
 5180: 5139,
 5289: 5146,
 5206: 5177,
 5215: 5352,
 5243: 5530,
 5738: 5310,
 5413: 5333,
 5373: 5534,
 5419: 5475,
 5521: 5441,
 55

In [53]:
get_sequence_positions('ecoli-genes-standard.ffn')

{190: 255,
 337: 2799,
 2801: 3733,
 3734: 5020,
 5234: 5530,
 6459: 5683,
 7959: 6529,
 8238: 9191,
 9306: 9893,
 10494: 9928,
 11356: 10643,
 11786: 11382,
 12163: 14079,
 14168: 15298,
 15445: 16557,
 16960: 16751,
 16903: 16751,
 17489: 18655,
 18715: 19620,
 20314: 19811,
 20508: 20233,
 21078: 20815,
 21181: 21399,
 21407: 22348,
 22391: 25207,
 25207: 25701,
 25826: 26275,
 26277: 27227,
 27293: 28207,
 28374: 29195,
 29651: 30799,
 30817: 34038,
 34300: 34695,
 35371: 34781,
 36162: 35377,
 37824: 36271,
 39115: 37898,
 40386: 39244,
 41931: 40417,
 42403: 43173,
 43188: 44129,
 44180: 45466,
 45463: 45750,
 45807: 47138,
 47246: 47776,
 47769: 49631,
 49823: 50302,
 51222: 50380,
 51606: 51229,
 52430: 51609,
 53416: 52427,
 54702: 53416,
 57109: 54755,
 57364: 58179,
 60346: 59687,
 63264: 60358,
 65780: 63429,
 66550: 65855,
 68337: 66835,
 70048: 68348,
 70387: 71265,
 71351: 72115,
 72927: 72229,
 74521: 72911,
 75480: 74497,
 77299: 75644,
 77388: 77519,
 77621: 78799,
 7

In [42]:
test_len=get_sequence_positions('out_test_full.txt')

In [43]:
len(test_len)

64097

#### (b) (3pts) Correctly predicted ORFs

In [58]:
%run correct_orfs.py out_test.txt ecoli-genes-standard.ffn

Number of orfs found: 4
Number of genes: 4212
Number of orfs correctly predicting genes: 0, (0.0 % of genes predicted)
Number of orfs correctly predicting a stop codon: 0, (0.0 % of stop     codons predicted)
Number of missing genes: 4212


In [59]:
%run orffinder.py ecoli-genome-sample.fna out_test.txt 50

[('frame 3', 'ATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAA', ' | :30-98 ')]
[('frame 2', 'ATGGTTTTTTCAATCATCGCCACCAGGTGGTTGGTGATTTTGGCGGGGGCAGAGAGGACGGTGGCCACCTGCCCCTGCCTGGCATTGCTTTCCAGAATATCGGCAACACGCAGAAAACGTTCTGCATTTGCCACTGATGTACCGCCGAACTTCAACACTCGCATGGTTGTTACCTCGTTACCTTTGGTCGAAAAAAAAAGCCCGCACTGTCAGGTGCGGGCTTTTTTCTGTGTTTCCTGTACGCGTCAGCCCGCACCGTTACCTGTGGTAATGGTGATGGTGGTGGTAATGGTGGTGCTAATGCGTTTCATGGATGTTGTGTACTCTGTAATTTTTATCTGTCTGTGCGCTATGCCTATATTGGTTAAAGTATTTAGTGACCTAAGTCAATAA', ' | :c500-108 '), ('frame 3', 'ATGTACCGCCGAACTTCAACACTCGCATGGTTGTTACCTCGTTACCTTTGGTCGAAAAAAAAAGCCCGCACTGTCAGGTGCGGGCTTTTTTCTGTGTTTCCTGTACGCGTCAGCCCGCACCGTTACCTGTGGTAA', ' | :c364-230 '), ('frame 3', 'ATGTTGTGTACTCTGTAA', ' | :c187-170 ')]


In [2]:
%run orffinder-lengths.py ecoli-genome-sample.fna out_test.txt 100

[]
[('frame 2', 'ATGGTTTTTTCAATCATCGCCACCAGGTGGTTGGTGATTTTGGCGGGGGCAGAGAGGACGGTGGCCACCTGCCCCTGCCTGGCATTGCTTTCCAGAATATCGGCAACACGCAGAAAACGTTCTGCATTTGCCACTGATGTACCGCCGAACTTCAACACTCGCATGGTTGTTACCTCGTTACCTTTGGTCGAAAAAAAAAGCCCGCACTGTCAGGTGCGGGCTTTTTTCTGTGTTTCCTGTACGCGTCAGCCCGCACCGTTACCTGTGGTAATGGTGATGGTGGTGGTAATGGTGGTGCTAATGCGTTTCATGGATGTTGTGTACTCTGTAATTTTTATCTGTCTGTGCGCTATGCCTATATTGGTTAAAGTATTTAGTGACCTAAGTCAATAA', ' | :c500-108'), ('frame 3', 'ATGTACCGCCGAACTTCAACACTCGCATGGTTGTTACCTCGTTACCTTTGGTCGAAAAAAAAAGCCCGCACTGTCAGGTGCGGGCTTTTTTCTGTGTTTCCTGTACGCGTCAGCCCGCACCGTTACCTGTGGTAA', ' | :c364-230')]


The orffinder doesn't find all the orfs and it's not very efficient, that's why I can't run it on the big dataset multiple times. 